# Projeto 2 - Ciência dos Dados

Nome: _Marco Tulio Masselli Rainho Teixeira_

Nome: _Talissa Gonçalves Albertini_

## Classificador automático de qualidade de investimentos em ações


### Descrição da Base de Dados 

A base de dados ultilizada neste projeto é composta por indicadores financeiros coletados ao longo de vários anos (1986-2020) da empresa Microsoft

Indicadores do DataSet Trimestral
* DATE: Dada de coleta das informações
* NET_INCOME: A diferença entre o lucro bruto e as despesas (em $10^6$ dólares)
* CF_FREE_CASH_FLOW: 
* CF_CASH_FROM_OPER:
* CF_CASH_FROM_INV_ACT:
* CF_CASH_FROM_FNC_ACT:
* CUR_RATIO:
* TOT_DEBT_TO_COM_EQY:
* RETURN_COM_EQY:

Indicadores do DataSet Diário
* DATE: Dada de coleta das informações
* CUR_MKT_CAP : Valor de mercado, valor de uma ação vezes o número total ed ações
* TURNOVER : Indica o quão rápido a empresa consegue arrecadar dinheito com o seu inventário
* PX LAST : Preço da ação 
* EQY_SH_OUT :
* PE RATIO :
* PX_TO_BOOK_RATIO :
* EQY_DPS :

### Técnica utilizada

Regressão logística multinomial

### Objetivo do Projeto

Este projeto tem o objetivo realizar a análise de indicadores financeiros de uma determinada ação do portfólio e responder a seguinte questão: Esta ação deve ser matida no portfólio ou não ? O critério relevante para responder essa questão é basicamente a probabilidade dessa ação valorizar durante o próximo trimestre, dado que parte significante dos indicadores são trimestrais.

In [10]:
# funcao para formatacao das ceulas

def disposicao_lateral(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    

### 1. Preparação do ambiente do Jupyter

In [11]:
%matplotlib inline
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from IPython.display import display_html

from sklearn import linear_model
from sklearn import metrics

# obs: descobrir por que as bibliotecas nao estao funcionando

#from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
#import plotly.graph_objs as go
#import plotly.plotly as py
#from plotly.graph_objs import *

### 2. Carregando a base de dados com os indicadores


In [12]:
xls = pd.ExcelFile('Dados_BBG.xlsx')
dft1 = pd.read_excel(xls, 'MSFT-Quarterly')
dft2 = pd.read_excel(xls, 'MSFT-Daily')

In [13]:
dft = dft1.iloc[6:133, 1:] # dft trimestral

dfd = dft2.iloc[6:, 1:] # dft diário

### 3. Tratamento do banco de dados de treinamento


In [8]:
# rectify the date columns
dft.iloc[:, 0] = dft.iloc[:, 0].dt.date 

dfd.iloc[:, 0] = dfd.iloc[:, 0].dt.date

In [9]:
# Set the column's names 

columns_dft = dft1.iloc[4,[1,3,5,7,9,11,13,15,17,19]].array  
dft.rename( columns={'Unnamed: 1':columns_dft[0]}, inplace=True )
dft.rename( columns={'Unnamed: 3':columns_dft[1]}, inplace=True )
dft.rename( columns={'Unnamed: 5':columns_dft[2]}, inplace=True )
dft.rename( columns={'Unnamed: 7':columns_dft[3]}, inplace=True )
dft.rename( columns={'Unnamed: 9':columns_dft[4]}, inplace=True )
dft.rename( columns={'Unnamed: 11':columns_dft[5]}, inplace=True )
dft.rename( columns={'Unnamed: 13':columns_dft[6]}, inplace=True )
dft.rename( columns={'Unnamed: 15':columns_dft[7]}, inplace=True )
dft.rename( columns={'Unnamed: 17':columns_dft[8]}, inplace=True )
dft.rename( columns={'Unnamed: 19':columns_dft[9]}, inplace=True )

columns_dfd = dft2.iloc[4,[1,3,5,7,9,11,13]].array  
dfd.rename( columns={'Unnamed: 1':columns_dfd[0]}, inplace=True )
dfd.rename( columns={'Unnamed: 3':columns_dfd[1]}, inplace=True )
dfd.rename( columns={'Unnamed: 5':columns_dfd[2]}, inplace=True )
dfd.rename( columns={'Unnamed: 7':columns_dfd[3]}, inplace=True )
dfd.rename( columns={'Unnamed: 9':columns_dfd[4]}, inplace=True )
dfd.rename( columns={'Unnamed: 11':columns_dfd[5]}, inplace=True )
dfd.rename( columns={'Unnamed: 13':columns_dfd[6]}, inplace=True )

NameError: name 'df1' is not defined

In [ ]:
# indicadores financeiros que serao analisados
print(columns_dft)
print(columns_dfd)

In [ ]:
# Get rid of unnamed columns thar remained
dft = dft.loc[:, ~dft.columns.str.contains('^Unnamed')]

dfd = dfd.loc[:, ~dfd.columns.str.contains('^Unnamed')]

In [ ]:
dfd.head()

In [ ]:
dft.head()

In [ ]:
# Rectify the data type
indicators_dft = ['NET_INCOME', 'CF_FREE_CASH_FLOW', 'CF_CASH_FROM_OPER', 'CF_CASH_FROM_INV_ACT', 'CF_CASH_FROM_FNC_ACT', 'CUR_RATIO',  'TOT_DEBT_TO_COM_EQY', 'RETURN_COM_EQY', 'EQY_DPS']
for indicator in indicators_dft:
    dft[indicator] = dft[indicator].astype(float)
    
dft['DATE'] = pd.to_datetime(dft['DATE'])



indicators_dfd = ['CUR_MKT_CAP', 'TURNOVER', 'PX_LAST', 'EQY_SH_OUT', 'PE_RATIO', 'PX_TO_BOOK_RATIO']
for indicator in indicators_dfd:
    dfd[indicator] = dfd[indicator].astype(float)
    
dfd['DATE'] = pd.to_datetime(dfd['DATE'])    

In [ ]:
# tipos de variavel
disposicao_lateral(dft.dtypes.to_frame(),dfd.dtypes.to_frame())

In [ ]:
# set date as index
dft = dft.set_index('DATE')
dfd = dfd.set_index('DATE')
dft.head()

In [ ]:
# converção de tabela diaria em trimestral e uniormizacao dos nomes dos df
dft1 = dft
dft2 = dfd.resample('Q').mean()

In [ ]:
# cria a coluna da variavel target , para ser usada na regressao logistica 
dft2['return'] = (dft2['PX_LAST'].diff()>0).astype('int')

In [ ]:
# junta os data frames 
dftmerged = pd.DataFrame()
dftmerged = dft1.merge(dft2, left_on='DATE', right_on='DATE')
dftmerged.head()

In [ ]:
 dft.describe().T

### 4. Análise exploratória dos dados

In [ ]:
# descricao dos df
# disposicao_lateral(dft1.describe().T, dft2.describe().T)

In [ ]:
# Skewness of the indicators
#dft1.skew()
#dft2.skew()
# disposicao_lateral(dft1.skew().to_frame(), dft2.skew().to_frame())

In [ ]:
# Null values
#dft1.isna().sum()
#dft2.isna().sum()

In [ ]:
# correlation table dft1
fig = plt.figure(figsize=(10,8))
sb.heatmap(dft1.corr(),annot=True,linewidths=.5);

In [ ]:
# correlation table dft2
fig = plt.figure(figsize=(8,5))
sb.heatmap(dft2.corr(),annot=True,linewidths=.5);

In [ ]:
# litsas de correlacao entre a variavel targef e as variaveis features

price_corr =  dftmerged.corr()['PX_LAST'].sort_values(ascending=False).to_frame()
price_corr

In [ ]:
# visualizaca dos indicadores no tempo
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(12,20))
dftmerged.plot(ax = axes[0],subplots=True);
# plot second pandas frame in subplot style
#dft2.plot(ax = axes[1],subplots=True)

In [ ]:
# adiciona a coluna return no dfd somente para alguma possivel comparacao com uma variavel nos graficos abaixo
dfd['return'] = (dfd['PX_LAST'].diff()>0).astype('int')

In [ ]:
# 2 scatter sobrepostos de dois indicadores, um scatter para os dias em que a acao valorizou, e outro para os quais ela desvalorizou
profit = dfd.loc[dfd['return'] == 1, :] 
loss = dfd.loc[dfd['return'] == 0, :]

plt.scatter(profit.loc[: , 'PE_RATIO'], profit.loc[: , 'CUR_MKT_CAP'], s=10, label='Profitble')
plt.scatter(loss.loc[:, 'PE_RATIO'], loss.loc[:, 'CUR_MKT_CAP'], s=10, label='Not profitable')
plt.legend()
plt.show()

In [ ]:
# loop para plotar o mesmo tipo de grafico da celula anterior, mas com um grafico para cada indicador

#for indicator in indicators_dfd: 
#    plt.scatter(profit.loc[: , indicator], profit.loc[: , 'PX_TO_BOOK_RATIO'], s=10, label='profitble')
#    plt.scatter(loss.loc[:, indicator], loss.loc[:, 'PX_TO_BOOK_RATIO'], s=10, label='Not profitable')
#    plt.legend()
#    plt.show()

In [ ]:
plt.figure(figsize=(12,6))
dfd['PX_LAST'].rolling(window=30).mean().plot(label='30 Day Avg')
dfd['PX_LAST'].plot(label='PX_LAST')
plt.legend()
# usar loc['2008-01-01':'2009-01-01'] qunado as colunas ja tiverem nomes

### 5. Classificador

In [ ]:
# funcao da regressao logistica multipla
def Classifier(df):
    target = df.loc[:,'return'].head()
    indicators = df.iloc[:,:-1].head()
    train_indicators, test_indicators, train_target, test_target = train_test_split(indicators, target, train_size=0.7)
    linear_model.LogisticRegression().fit(train_indicators, train_target)
    # retorna valores de test_target, baseado nos valores de test_indicators
    return LogisticRegression().predict(test_indicators)

In [ ]:
# obs: a funcao nao esta funcionando pq falta arrumar os NaN no dfmerged
# possibilidades: atribuir valores de medias ?, regressao linear ?

In [ ]:
# teste de acuracia do classificador

LogisticRegression().score(test_indicators, test_target)

In [ ]:
# bobliografia 

# regressao logistica: https://dataaspirant.com/2017/05/15/implement-multinomial-logistic-regression-python/ 